# 4bit 양자화(QLoRA) 구성으로 LLM 모델 파인튜닝

## RunPod 셋팅

Pod Template 에서 제공하는 Unsloth/unsloth:latest Docker Image 기반을 사용합니다.  
강사의 안내에 따라 적절하게 환경 셋팅 후 실습을 진행해주시기 바랍니다.

## 파인튜닝 과정 소개

OpenAI GPT-OSS 20B 모델을 4bit 양자화(QLoRA) 구성으로 파인튜닝하는 전체 과정을 살펴보고,  
원격지에 있는 서버의 Python Kernel 에 현재 Jupyter Notebook 을 연동하여 확인합니다.

In [1]:
# 원격지 서버의 GPU 정보 확인
!nvidia-smi

Sun Nov 16 13:37:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:BE:00.0 Off |                    0 |
| N/A   33C    P0             74W /  700W |       1MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 1024  # Context length - 더 길게해도 되지만, 메모리를 많이 쓰게 됩니다.
dtype = None

# 사전 양자화 모델 목록 (모델 다운로드 + 현실적인 파인튜닝 시간 소요 = 4Bit 진행)
fourbit_models = [
    "unsloth/gpt-oss-20b",  # MXFP4 형식 - 인퍼런스 용도
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit",  # bitsandbytes 4bit 양자화
    "unsloth/gpt-oss-20b-BF16",  # Bfloat16
]  # 전체 목록은 https://huggingface.co/collections/unsloth/gpt-oss 참고

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gpt-oss-20b",
    dtype=dtype,  # Auto Data Type 선택
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # 4bit 양자화 로딩 = True
    full_finetuning=False,  # 전체 파인튜닝이 필요한 경우
    # token = "hf_...",  # Access 필요한 모델 접근 시 사용
)

---

## GPT-OSS-20B 아키텍처 검증

모델이 제대로 로딩되었는지, 그리고 어떤 활성화 함수를 사용하는지 확인합니다.

#### 확인 항목
1. **활성화 함수**: `hidden_act` → "silu" (Swish) 확인
2. **SwiGLU 설정**: `swiglu_limit` 파라미터 존재 확인
3. **MoE 구조**: 전문가 수와 활성 전문가 수 확인
4. **FFN 구조**: gate_proj + up_proj + down_proj 조합 (SwiGLU 전형적 구조)

#### 참고 자료
- **SwiGLU 원본 논문**: [GLU Variants Improve Transformer](https://arxiv.org/abs/2002.05202) (Shazeer, 2020)
- **Swish 활성화**: [Searching for Activation Functions](https://arxiv.org/abs/1710.05941) (Ramachandran et al., 2017)
- **아키텍처 분석**: [Sebastian Raschka - From GPT-2 to gpt-oss](https://magazine.sebastianraschka.com/p/from-gpt-2-to-gpt-oss-analyzing-the)
- **모델카드**: [gpt-oss-20B & gpt-oss-120B](https://arxiv.org/abs/2508.10925)

In [3]:
# GPT-OSS-20B 아키텍처 검증
# 1. 기본 정보
print(f"\n모델 이름: {model.config.model_type}")
print(f"총 레이어 수: {model.config.num_hidden_layers}")
print(f"Hidden 차원: {model.config.hidden_size}")
print(f"Attention 헤드: {model.config.num_attention_heads}")

# 2. 활성화 함수 확인
print(f"\n활성화 함수: {model.config.hidden_act}")
if model.config.hidden_act == "silu":
    print("SiLU (= Swish) 활성화 함수 사용 확인!")
    print("  → Swish(x) = x · σ(x)")

# 3. SwiGLU 설정 확인
if hasattr(model.config, "swiglu_limit"):
    print(f"SwiGLU limit: {model.config.swiglu_limit}")
    print("SwiGLU 파라미터 존재 확인!")
    print("SwiGLU(gate, up) = Swish(gate) ⊙ up")

# 4. FFN 구조 확인
print("\nFFN 구조:")
print(f"Intermediate size: {model.config.intermediate_size}")
print(f"Hidden size: {model.config.hidden_size}")
print("구성: gate_proj + up_proj → SwiGLU → down_proj")

# 5. MoE (Mixture of Experts) 정보
print("\nMoE 구조:")
print(f"   - 전체 전문가 수: {model.config.num_local_experts}")
print(f"   - 토큰당 활성 전문가: {model.config.num_experts_per_tok}")
print(
    f"   - 활성 파라미터 비율: {model.config.num_experts_per_tok / model.config.num_local_experts * 100:.1f}%"
)

# 6. 컨텍스트 길이
print("\n컨텍스트 길이:")
print(f"   - 최대 위치 임베딩: {model.config.max_position_embeddings:,} 토큰")
print(f"   - 초기 컨텍스트: {model.config.initial_context_length:,} 토큰")
print(f"   - RoPE Scaling: {model.config.rope_scaling['rope_type']}")

# 7. 양자화 정보
if hasattr(model.config, "quantization_config"):
    print("\n양자화 설정:")
    print(f"   - 방식: {model.config.quantization_config}")
    print("  MXFP4 양자화 적용됨")


모델 이름: gpt_oss
총 레이어 수: 24
Hidden 차원: 2880
Attention 헤드: 64

활성화 함수: silu
SiLU (= Swish) 활성화 함수 사용 확인!
  → Swish(x) = x · σ(x)
SwiGLU limit: 7.0
SwiGLU 파라미터 존재 확인!
SwiGLU(gate, up) = Swish(gate) ⊙ up

FFN 구조:
Intermediate size: 2880
Hidden size: 2880
구성: gate_proj + up_proj → SwiGLU → down_proj

MoE 구조:
   - 전체 전문가 수: 32
   - 토큰당 활성 전문가: 4
   - 활성 파라미터 비율: 12.5%

컨텍스트 길이:
   - 최대 위치 임베딩: 131,072 토큰
   - 초기 컨텍스트: 4,096 토큰
   - RoPE Scaling: yarn


---
## LoRA 어댑터 추가

LoRA는 전체 파라미터 중 약 1%만 학습하도록 만들어 GPU 메모리를 크게 절약하며, QLoRA 구성에서는 4bit로 압축된 베이스 모델 위에 이 어댑터를 덧붙여 효율적으로 파인튜닝을 진행합니다.

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # 0보다 큰 임의의 수 선택 가능 (권장: 8, 16, 32, 64, 128)
    target_modules=[
        # Attention
        "q_proj",  # Query Projection
        "k_proj",  # Key Projection
        "v_proj",  # Value Projection
        "o_proj",  # Output Projection
        # FFN(Feed-Forward Network)
        "gate_proj",  # Gate Projection
        "up_proj",  # Up Projection
        "down_proj",  # Down Projection
    ],
    lora_alpha=16,
    lora_dropout=0,  # 임의 값 지원, 하지만 0일 때 최적화됨
    bias="none",  # 임의 값 지원, 하지만 "none"일 때 최적화됨
    use_gradient_checkpointing="unsloth",  # 긴 컨텍스트를 위해 True 또는 "unsloth" 사용
    random_state=3407,
    use_rslora=False,  # Rank Stabilized LoRA 지원
    loftq_config=None,  # LoftQ 지원
)

Unsloth: Making `model.base_model.model.model` require gradients


---

## LoRA 하이퍼파라미터 가이드

이 섹션은 모든 모델(Llama, Qwen, Gemma, Mistral, GPT-OSS 등)에 적용 가능한 일반적인 LoRA 설정 가이드입니다.

### 핵심 하이퍼파라미터 요약표

| 파라미터 | 권장값 | 설명 |
|---------|--------|------|
| **Learning Rate** | `2e-4` (SFT), `5e-6` (RL) | 학습 속도 조절 |
| **Epochs** | `1-3` | 전체 데이터셋 반복 횟수 |
| **LoRA Rank (r)** | `16` 또는 `32` | 어댑터 행렬 차원 (8, 16, 32, 64, 128) |
| **LoRA Alpha** | `r` 또는 `2*r` | 스케일링 팩터 (보통 16 또는 32) |
| **LoRA Dropout** | `0` | 정규화 (기본값 0, 필요시 0.1) |
| **Weight Decay** | `0.01` | 가중치 페널티 (0.01 ~ 0.1) |
| **Warmup Steps** | 전체의 `5-10%` | 학습률 워밍업 단계 |
| **Scheduler** | `linear` 또는 `cosine` | 학습률 스케줄러 |
| **Target Modules** | 아래 참조 | 적용할 레이어 선택 |

---

### 대부분 요즘의 모델들 = Target Modules 가이드

```python
target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",  # Attention
    "gate_proj", "up_proj", "down_proj",      # FFN (SwiGLU)
]
```

---

### LoRA Alpha와 Rank의 관계

#### 기본 공식
```python
effective_scaling = lora_alpha / r
```

#### 권장 설정
- **표준**: `lora_alpha = r` → 스케일링 = 1.0
- **적극적 학습**: `lora_alpha = 2 * r` → 스케일링 = 2.0
- **Rank-Stabilized LoRA (RSLoRA)**: `use_rslora=True` → 스케일링 = `alpha / sqrt(r)`

#### 예시
```python
# 설정 1: 보수적 학습
r = 16
lora_alpha = 16  # 스케일링 = 1.0

# 설정 2: 적극적 학습 (권장)
r = 16
lora_alpha = 32  # 스케일링 = 2.0

# 설정 3: 고차원 안정화
r = 128
lora_alpha = 128
use_rslora = True  # 스케일링 = 128/sqrt(128) ≈ 11.3
```

---

### Epochs와 Learning Rate 선택

#### Learning Rate 가이드

| 작업 유형 | Learning Rate | 설명 |
|----------|---------------|------|
| **일반 파인튜닝 (SFT)** | `2e-4` (0.0002) | 기본 시작점 |
| **강화학습 (GRPO, DPO)** | `5e-6` (0.000005) | 안정적인 정책 학습 |
| **Full Fine-tuning** | `1e-5` ~ `5e-5` | 더 낮은 학습률 필요 |
| **도메인 적응** | `1e-4` | 중간 학습률 |

#### Epochs 가이드
- **1 Epoch**: 대규모 데이터셋 (>10K 샘플)
- **2-3 Epochs**: 표준 설정 (1K-10K 샘플)
- **3+ Epochs**: 작은 데이터셋 (<1K 샘플) - 과적합 주의!

---

### LoRA vs QLoRA 비교

| 특성 | LoRA (16-bit) | QLoRA (4-bit) |
|------|---------------|---------------|
| **정확도** | 약간 더 높음 | 약간 낮음 (1-2% 차이) |
| **속도** | 빠름 | 약간 느림 |
| **VRAM 사용량** | 4배 더 많음 | 4배 적음 |
| **70B 모델** | ~180GB VRAM | **~45GB VRAM** |
| **권장 상황** | GPU 메모리 충분, 최고 성능 | VRAM 제한적, 대형 모델 |

#### 실사용 예시
```python
# QLoRA: 4-bit 양자화 (메모리 효율)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.3-70B-Instruct",
    load_in_4bit = True,  # QLoRA
    dtype = None,
)

# LoRA: 16-bit (성능 우선)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.1-8B",
    load_in_4bit = False,  # Full 16-bit
    dtype = torch.bfloat16,
)
```

---

### 과적합(Overfitting) vs 과소적합(Underfitting) 진단

#### 과적합 증상
- 훈련 손실 매우 낮음
- 검증 손실 높음 또는 증가
- 실제 추론 시 훈련 데이터 그대로 외워서 출력
- 새로운 질문에 일반화 실패

**해결책**:
```python
# 1. Epochs 줄이기
num_train_epochs = 1  # 3 → 1

# 2. Dropout 추가
lora_dropout = 0.05  # 0 → 0.05

# 3. Weight Decay 증가
weight_decay = 0.01  # 0.001 → 0.01

# 4. 데이터 증강
# 더 많은 다양한 샘플 추가
```

#### 과소적합 증상
- 훈련 손실 높음
- 검증 손실도 높음
- 모델이 기본 패턴도 학습 못함

**해결책**:
```python
# 1. Learning Rate 증가
learning_rate = 5e-4  # 2e-4 → 5e-4

# 2. Epochs 증가
num_train_epochs = 3  # 1 → 3

# 3. LoRA Rank 증가
r = 32  # 16 → 32
lora_alpha = 64  # 32 → 64

# 4. Target Modules 확장
# 모든 레이어 포함
```

---

### Unsloth 최적화 팁

#### Gradient Checkpointing
```python
use_gradient_checkpointing = "unsloth"  # 메모리 30% 절약
# True: 표준 체크포인팅
# "unsloth": Unsloth 최적화 (긴 컨텍스트 지원)
```

#### 배치 사이즈와 Gradient Accumulation
```python
# 실질적 배치 사이즈 = per_device_batch_size × gradient_accumulation_steps × num_gpus

# 예시 1: 작은 VRAM (16GB)
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
# → Effective Batch Size = 4

# 예시 2: 큰 VRAM (80GB)
per_device_train_batch_size = 4
gradient_accumulation_steps = 2
# → Effective Batch Size = 8
```

#### Optimizer 선택
```python
# 추천: 8-bit AdamW (메모리 효율)
optim = "adamw_8bit"

# 대안:
# optim = "adamw_torch"      # 표준 AdamW
# optim = "paged_adamw_8bit"  # 페이징 최적화
```

---

### 참고 자료
- **LoRA 논문**: [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- **QLoRA 논문**: [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)
- **RSLoRA 논문**: [Rank-Stabilized LoRA](https://arxiv.org/abs/2312.03732)
- **Unsloth 공식 문서**: [LoRA Hyperparameters Guide](https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/lora-hyperparameters-guide)

---

## 데이터셋 준비 가이드

파인튜닝의 성공은 데이터셋 품질에 달려있습니다.  
이 섹션은 모든 모델에 적용 가능한 데이터셋 준비 가이드입니다.

### 데이터 포맷 종류

#### 1. Continued Pretraining (연속 사전학습)
원시 텍스트를 그대로 사용 (구조 없음):

```json
{
  "text": "양자역학은 원자와 아원자 입자의 행동을 설명하는 물리학 분야입니다. 1900년대 초 맥스 플랑크의 흑체 복사 연구에서 시작되었으며..."
}
```

**사용 사례**: 도메인 지식 주입, 언어 적응

---

#### 2. Alpaca Format (단일 턴 instruction)
Instruction + Input + Output 구조:

```json
{
  "instruction": "주어진 텍스트를 영어로 번역하세요.",
  "input": "안녕하세요, 반갑습니다.",
  "output": "Hello, nice to meet you."
}
```

**사용 사례**: 
- 번역, 요약, 분류 태스크
- 명확한 입출력 관계
- 단일 응답 생성

---

#### 3. ShareGPT Format (다중 턴 대화)
`from`/`value` 구조로 대화 표현:

```json
{
  "conversations": [
    {"from": "human", "value": "파스타 카르보나라 만드는 법 알려줘"},
    {"from": "gpt", "value": "전통 로마식 레시피와 간단한 버전 중 어떤 걸 원하세요?"},
    {"from": "human", "value": "전통 로마식으로 알려줘"},
    {"from": "gpt", "value": "로마식 카르보나라는 파스타, 구안치알레, 계란, 페코리노 치즈, 후추만 사용합니다..."}
  ]
}
```

**사용 사례**: 
- 챗봇 개발
- 문맥 유지 대화
- 다단계 추론

---

#### 4. ChatML Format (OpenAI 표준)
`role`/`content` 구조 (Hugging Face 기본):

```json
{
  "messages": [
    {"role": "system", "content": "당신은 한국어 요리 전문가입니다."},
    {"role": "user", "content": "김치찌개 레시피 알려줘"},
    {"role": "assistant", "content": "김치찌개 재료는 다음과 같습니다: 신김치 1컵..."}
  ]
}
```

**사용 사례**: 
- OpenAI API 호환
- System prompt 활용
- 역할 기반 대화

---

### Unsloth 데이터셋 포맷팅

#### Step 1: 지원 템플릿 확인
```python
from unsloth.chat_templates import CHAT_TEMPLATES

print(list(CHAT_TEMPLATES.keys()))
# 출력 예시: ['llama-3.3', 'gemma-3', 'qwen2.5', 'phi-4', 'chatml', ...]
```

#### Step 2: Chat Template 적용
```python
from unsloth.chat_templates import get_chat_template

# 예시 1: Llama 3.3
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.3",
)

# 예시 2: Qwen 2.5
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen2.5",
)

# 예시 3: Gemma 3
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
```

#### Step 3: 포맷팅 함수 정의

##### ShareGPT 스타일
```python
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, 
            tokenize=False, 
            add_generation_prompt=False
        ) 
        for convo in convos
    ]
    return {"text": texts}
```

##### ChatML 스타일
```python
def formatting_prompts_func(examples):
    messages = examples["messages"]
    texts = [
        tokenizer.apply_chat_template(
            msg, 
            tokenize=False, 
            add_generation_prompt=False
        ) 
        for msg in messages
    ]
    return {"text": texts}
```

##### Alpaca 스타일
```python
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        message = [
            {"role": "user", "content": f"{instruction}\n{input_text}"},
            {"role": "assistant", "content": output}
        ]
        text = tokenizer.apply_chat_template(message, tokenize=False)
        texts.append(text)
    
    return {"text": texts}
```

#### Step 4: 데이터셋 로드 및 적용
```python
from datasets import load_dataset

# Hugging Face 데이터셋 로드
dataset = load_dataset("your-username/dataset-name", split="train")

# 포맷팅 적용
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

# 결과 확인
print(dataset[0]["text"])
```

---

### 데이터셋 크기 가이드

| 데이터셋 크기 | 권장 Epochs | 주의사항 |
|--------------|------------|----------|
| **< 100 샘플** | 3-5 | 합성 데이터 추가 고려 |
| **100-1,000** | 2-3 | 최소 권장 크기 |
| **1,000-10,000** | 1-2 | 표준 설정 |
| **10,000+** | 1 | 과적합 위험 낮음 |

#### 데이터 품질 > 데이터 양
- **좋은 예시**: 정확하고 다양한 1,000개 샘플
- **나쁜 예시**: 중복되고 오류 많은 10,000개 샘플

---

### 합성 데이터 생성

데이터가 부족할 때 LLM으로 데이터 증강이 가능합니다.  
(물론 비용을 적절하게 고려해서 LLM API 또는 OpenSource Model 선택 필요)

**Unsloth 공식 예제 - 합성 데이터 노트북**: [Synthetic Data Generation](https://colab.research.google.com/drive/1njCCbE1YVal9xC83hjdo2hiGItpY_D6t)

---

### 추론 모델 데이터셋 구조

추론 능력(Thinking, Reasoning)이 있는 모델을 파인튜닝할 때:

#### SFT용 (추론 과정 포함)
```json
{
  "messages": [
    {
      "role": "user", 
      "content": "x^5 + 3x^4 - 10 = 3을 푸시오"
    },
    {
      "role": "assistant", 
      "content": "<reasoning>\n1. 방정식 정리: x^5 + 3x^4 - 13 = 0\n2. 근의 공식 적용 불가, 수치해법 필요\n3. Newton-Raphson 방법 사용...\n</reasoning>\n\n<answer>\nx ≈ 1.563\n</answer>"
    }
  ]
}
```

#### GRPO/RL용 (답만 포함)
```json
{
  "prompt": [
    {"role": "user", "content": "x^5 + 3x^4 - 10 = 3을 푸시오"}
  ],
  "answer": "x ≈ 1.563"
}
```

---

### 데이터셋 FAQ

#### Q: 여러 데이터셋을 합칠 수 있나요?
**A**: 네, 두 가지 방법이 있습니다:

```python
# 방법 1: 데이터셋 결합
from datasets import concatenate_datasets

dataset1 = load_dataset("dataset1", split="train")
dataset2 = load_dataset("dataset2", split="train")
combined = concatenate_datasets([dataset1, dataset2])

# 방법 2: Multiple Datasets 노트북 사용
# https://colab.research.google.com/drive/1njCCbE1YVal9xC83hjdo2hiGItpY_D6t
```

#### Q: 같은 모델을 여러 번 파인튜닝할 수 있나요?
**A**: 가능하지만 비권장. 대신 모든 데이터를 합쳐서 한 번에 학습하세요.

```python
# 비권장: 순차적 파인튜닝
model_v1 = finetune(base_model, dataset1)
model_v2 = finetune(model_v1, dataset2)  # 이전 지식 손실 가능

# 권장: 통합 학습
combined_dataset = concatenate_datasets([dataset1, dataset2])
model = finetune(base_model, combined_dataset)
```

---

### 데이터셋 활용 가이드

1. **데이터 검증**: 학습 전 반드시 샘플 10개 이상 수동 검토
2. **균형 유지**: 클래스/카테고리별 샘플 수 균등하게
3. **중복 제거**: 동일하거나 유사한 샘플 제거
4. **길이 제한**: `max_seq_length` 내에 대부분 샘플이 들어오도록
5. **검증 세트**: 전체의 10% 정도는 검증용으로 분리

---

### LoRA 설정 상세 가이드

#### 1️⃣ target_modules: 왜 이 7개 모듈을 선택했는가?

LoRA(Low-Rank Adaptation)는 대형 언어 모델의 **모든 가중치를 업데이트하는 대신**, 특정 선형 레이어(linear projection)에만 저차원 행렬을 추가하여 효율적으로 파인튜닝하는 기법입니다.

선택된 7개 모듈은 **Transformer 블록의 핵심 연산**을 담당하며, GPT-OSS-20B/Qwen 아키텍처에서 가장 중요한 표현 학습이 일어나는 지점입니다.

---

#### 2️⃣ Transformer 아키텍처: 각 모듈의 역할

#### Multi-Head Self-Attention (4개 모듈)

```
입력 X (d_model 차원)
    ↓
┌─────────────────────────────────────┐
│  q_proj: X → Q (Query)              │  "이 토큰이 찾는 정보"
│  k_proj: X → K (Key)                │  "이 토큰이 제공하는 정보"
│  v_proj: X → V (Value)              │  "실제 전달될 정보"
└─────────────────────────────────────┘
    ↓
Attention(Q, K, V) = softmax(QK^T/√d) × V
    ↓
┌─────────────────────────────────────┐
│  o_proj: Attention → Output         │  "어텐션 결과를 원래 차원으로"
└─────────────────────────────────────┘
```

- **q_proj (Query)**: "현재 토큰이 **어떤 정보를 찾고 싶은지**" 결정
  - 예: "날씨" 토큰 → "장소, 시간 정보를 찾겠다"는 쿼리 생성
  
- **k_proj (Key)**: "각 토큰이 **어떤 정보를 제공할 수 있는지**" 표현
  - 예: "서울" 토큰 → "나는 장소 정보야"라는 키 생성
  
- **v_proj (Value)**: "실제로 **전달될 의미 정보**" 저장
  - 예: "서울" 토큰 → 도시, 수도, 지역 등의 의미 벡터
  
- **o_proj (Output)**: 여러 헤드의 어텐션 결과를 **결합하여 원래 차원으로 변환**
  - Multi-head 결과를 통합해 다음 레이어로 전달

#### Feed-Forward Network (3개 모듈)

```
Attention 출력
    ↓
┌─────────────────────────────────────┐
│  gate_proj: X → Gate (SwiGLU 활성화) │  "어떤 정보를 통과시킬지"
│  up_proj:   X → Up (차원 확장)       │  "정보 표현력 증가 (4d → 16d)"
└─────────────────────────────────────┘
    ↓
SwiGLU(gate, up) = gate × σ(gate) ⊙ up
    ↓
┌─────────────────────────────────────┐
│  down_proj: Hidden → Output         │  "다시 원래 차원으로 (16d → 4d)"
└─────────────────────────────────────┘
```

- **gate_proj**: SwiGLU 활성화 함수의 **게이트 역할**
  - "이 정보는 중요하니 통과, 저건 덜 중요하니 차단"
  
- **up_proj**: Hidden Dimension 을 **4배 확장** (예: 4096 → 16384)
  - 더 풍부한 비선형 변환 공간 제공
  
- **down_proj**: 확장된 차원을 다시 **원래 크기로 축소**
  - 학습된 고차원 표현을 압축해 다음 레이어로 전달

---

#### 3️⃣ LoRA 이론: 왜 이 모듈들에만 적용하는가?

#### LoRA의 핵심 아이디어

일반 파인튜닝:
```
W' = W₀ + ΔW
- W₀: 사전학습 가중치 (frozen, 4096×4096)
- ΔW: 학습할 변화량 (full rank, 4096×4096 = 16M 파라미터)
```

LoRA 파인튜닝:
```
W' = W₀ + BA
- W₀: 사전학습 가중치 (frozen)
- B: 저차원 행렬 (4096×8)
- A: 저차원 행렬 (8×4096)
- 학습 파라미터: (4096×8 + 8×4096) = 65K (256배 감소!)
```

#### Module 선택 기준: "정보 병목" 지점 공략

1. **Attention 모듈 (q/k/v/o_proj)**
   - **토큰 간 관계 학습**의 핵심
   - 의도 분류에서 "질문/명령/진술"을 구분하는 **문맥 패턴** 학습
   - 예: "~해줘" (명령) vs "~할까?" (질문) 차이는 어텐션으로 포착

2. **FFN 모듈 (gate/up/down_proj)**
   - **도메인 특화 지식** 저장소
   - 한국어 의도 표현 (종결어미, 높임법, 억양) 학습
   - 예: "~요" (공손), "~ㅂ니다" (격식체) 패턴 인코딩

3. **왜 Embedding/LN은 제외?**
   - **Embedding**: 토큰 → 벡터 변환 (단어장 변화 없으면 불필요)
   - **LayerNorm**: 정규화만 담당 (표현력 낮음)
   - **LoRA는 "선형 투영"에만 효과적** (Hu et al., 2021)

---

#### 4️⃣ 의도 분류 태스크에 최적화

#### 한국어 의도 특성과 모듈 매칭

| 의도 특성 | 학습 모듈 | 역할 |
|----------|----------|------|
| **종결어미** ("~해줘", "~할까요") | **gate_proj, up_proj** | 어미 패턴 → 의도 매핑 |
| **문맥 의존** ("이거 어때?" vs "이거 어때!") | **q_proj, k_proj** | 앞뒤 토큰 관계 파악 |
| **높임법** ("주세요" vs "줘") | **v_proj, o_proj** | 공손도 임베딩 학습 |
| **억양 단서** (물음표, 느낌표) | **down_proj** | 비선형 패턴 인식 |

#### 7개 모듈만 따로 적용했을 때의 장점(Lora 장점과 통합)
 
**표현력**: Attention + FFN 전체를 커버해 복잡한 의도 패턴 학습  
**효율성**: 전체 모델의 ~1%만 학습 (메모리 절약)  
**안정성**: 검증된 조합 (Qwen, Llama, GPT 공통 설정)  
**확장성**: 다른 도메인(감정 분석, 요약)도 동일 설정 재사용 가능

---

#### 5️⃣ 파라미터 설정 상세

- **r (LoRA Rank) = 8**
  - 저차 행렬의 차원 (작을수록 메모리↓, 클수록 표현력↑)
  - 의도 분류는 7개 클래스로 단순 → rank 8로 충분
  - 계산: 7개 모듈 × (4096×8 + 8×4096) × 2(B+A) = ~0.9M 파라미터

- **lora_alpha = 16**
  - 스케일링 팩터 (α/r = 16/8 = 2)
  - LoRA 논문 권장값: α = 2r
  - 작은 rank에서도 안정적 업데이트 보장

- **lora_dropout = 0**
  - 드롭아웃 비활성화 (메모리 효율 최대화)
  - 데이터 55K → 충분히 크므로 과적합 위험 낮음

- **bias = "none"**
  - 바이어스 항 제외 (추가 파라미터 없음)

- **use_gradient_checkpointing = "unsloth"**
  - Unsloth 최적화 체크포인팅
  - VRAM 30% 절약, 128K 컨텍스트 지원

- **use_rslora / loftq_config = False**
  - 고급 기법 비활성화 (기본 QLoRA 유지)
  - RSLoRA: Rank-Stabilized LoRA (rank 증가 시 유용)
  - LoftQ: 양자화 인식 LoRA (고급 최적화)

---

### Reasoning Effort (추론 강도)
OpenAI `gpt-oss` 모델은 "reasoning effort"를 조절하여 추론 품질과 응답 지연(latency) 사이에서 원하는 균형을 잡을 수 있게 해 줍니다. 생각에 사용할 토큰 수가 많을수록 더 깊은 추론이 가능하지만 시간이 더 걸립니다.

선택 가능한 세 단계는 다음과 같습니다.

* **Low**: 속도가 가장 빠른 대신 복잡한 다단계 추론에는 적합하지 않은 모드
* **Medium**: 속도와 품질을 모두 고려한 균형형 모드
* **High**: 가장 강력한 추론 성능을 제공하지만 토큰 사용량과 지연 시간이 증가하는 모드

In [5]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="low",  # low, medium, high (세가지 설정 가능)
).to("cuda")

_ = model.generate(**inputs, max_new_tokens=64, streamer=TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-11-16

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>Equation: x^5+3x^4-10=3 => x^5+3x^4-13=0. Solve real roots? Likely factor? Try integer: x=1 gives 1+3-13=-9. x=2 gives 32+48-


In [22]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_sharegpt

raw_ds = load_dataset("wicho/kor_3i4k", split="train")

# ClassLabel 이면 names가 있고, 아니면 그냥 None
label_feature = raw_ds.features["label"]
if hasattr(label_feature, "names"):
    id2label = {i: name for i, name in enumerate(label_feature.names)}
else:
    # 혹시 ClassLabel이 아니라면, 그냥 str로 캐스팅
    id2label = None

# Harmony 템플릿/레이블 검증
from collections import Counter

REQUIRED_TOKENS = [
    "<|start|>system",
    "<|start|>user<|message|>",
    "<|start|>assistant<|channel|>final<|message|>",
    "<|return|>",
]

issues = []
intent_counter = Counter()

for idx, example in enumerate(raw_ds):
    text = example["text"]
    for token in REQUIRED_TOKENS:
        if token not in text:
            issues.append((idx, f"missing token: {token}"))
            break
    # 의도 라벨 유무 확인 (assistant 응답 끝에 위치)
    intent_text = text.split("<|start|>assistant<|channel|>final<|message|>")[-1]
    label = intent_text.replace("<|return|>", "").strip()
    if not label:
        issues.append((idx, "missing intent label"))
    else:
        intent_counter[label] += 1

print(f"검증 완료: 총 {len(raw_ds)}개 샘플")
print(f"의도 분포 상위 5개: {intent_counter.most_common(5)}")
if issues:
    print(f"발견된 이슈 {len(issues)}건 (최초 5개): {issues[:5]}")
else:
    print("모든 샘플이 요구 토큰과 의도 라벨을 포함합니다.")


def kor3i4k_to_sharegpt(examples):
    convos = []

    for text, label in zip(examples["text"], examples["label"], strict=False):
        if id2label is not None:
            label_str = id2label[int(label)]
        else:
            label_str = str(label)

        # 필요하면 kor_3i4k 전체 label 목록도 instruction에 넣을 수 있음
        # 여기선 간단히 "label_str만 출력하라"고 강하게 제한
        user_msg = (
            "다음 한국어 발화의 화자 의도(label)를 분류해 주세요.\n"
            "정답으로는 해당 label 이름만 한 단어로 출력하세요.\n\n"
            f"발화: {text}"
        )

        assistant_msg = label_str

        convos.append(
            [
                {"from": "human", "value": user_msg},
                {"from": "gpt", "value": assistant_msg},
            ]
        )

    return {"conversations": convos}


sharegpt_ds = raw_ds.map(
    kor3i4k_to_sharegpt,
    batched=True,
    remove_columns=raw_ds.column_names,  # text/label 제거하고 conversations만 남길지 선택
)

dataset = standardize_sharegpt(sharegpt_ds)


def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {
        "text": texts,
    }


dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)


def add_channel_final(examples):
    fixed_texts = []
    for text in examples["text"]:
        fixed_texts.append(
            text.replace(
                "<|start|>assistant<|message|>",
                "<|start|>assistant<|channel|>final<|message|>",
            )
        )
    return {"text": fixed_texts}


# 이미 채널 태그가 있으면 중복으로 안 바꾸도록 한 번만 체크
if "<|channel|>final" not in dataset[0]["text"]:
    dataset = dataset.map(add_channel_final, batched=True)
dataset

검증 완료: 총 55134개 샘플
의도 분포 상위 5개: [('죄송합니다', 23), ('안녕하세요', 22), ('그럼', 17), ('감사합니다', 17), ('내가', 15)]
발견된 이슈 55134건 (최초 5개): [(0, 'missing token: <|start|>system'), (1, 'missing token: <|start|>system'), (2, 'missing token: <|start|>system'), (3, 'missing token: <|start|>system'), (4, 'missing token: <|start|>system')]


Map:   0%|          | 0/55134 [00:00<?, ? examples/s]

Map:   0%|          | 0/55134 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations', 'text'],
    num_rows: 55134
})

In [23]:
from trl import SFTConfig, SFTTrainer

# NOTE: 여기 설정은 완전 경험의 영역입니다.
# 정확히 어떤 설정이 어떻게 학습에 맞아들어가는지는 모델을 많이 훈련해본 경험이 있으신 분들만 알 수 있습니다.
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=SFTConfig(
        per_device_train_batch_size=1000,
        gradient_accumulation_steps=4,
        warmup_steps=3,
        num_train_epochs=1,  # 전체 훈련 1회 실행을 위해 설정
        # max_steps=None,  # max_steps를 설정하면 num_train_epochs를 None으로 두거나 주석 처리
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to=None,
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=64):   0%|          | 0/55134 [00:00<?, ? examples/s]

In [24]:
# Unsloth의 `train_on_completions` 메서드를 사용하여 어시스턴트 출력에만 학습하고 사용자 입력에 대한 손실은 무시합니다.
# 이렇게 하면 파인튜닝의 정확도가 향상되고 손실도 감소합니다!

from unsloth.chat_templates import train_on_responses_only

gpt_oss_kwargs = dict(
    instruction_part="<|start|>user<|message|>",
    response_part="<|start|>assistant<|channel|>final<|message|>",
)

trainer = train_on_responses_only(trainer, **gpt_oss_kwargs)

Map (num_proc=64):   0%|          | 0/55134 [00:00<?, ? examples/s]

In [25]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 80GB HBM3. Max memory = 79.209 GB.
39.785 GB of memory reserved.


모델을 훈련시켜봅시다!  
만약 멈추게 되었을 때, 훈련을 재개하려면 `trainer.train(resume_from_checkpoint = True)`를 설정하세요.  
(Checkpoints_N) 폴더가 존재해야합니다.

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

---

### 결과물 확인을 위한 모델 추론 (Inference)
모델을 실행해봅시다! instruction과 input을 변경할 수 있으며, output은 비워두세요!

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant.",
    },
    {"role": "user", "content": ""},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="medium",
).to("cuda")

from transformers import TextStreamer

_ = model.generate(**inputs, max_new_tokens=64, streamer=TextStreamer(tokenizer))

---

### 모델 저장 및 로딩
최종 모델을 LoRA 어댑터 형태로 저장하려면 Hugging Face Hub(`push_to_hub`)나 로컬(`save_pretrained`)에 보관하세요.

**주의**: 현재 시점에서는 Unsloth를 통해서만 파인튜닝 가중치를 직접 불러올 수 있으며, Unsloth 라이브러리에서 vLLM·GGUF 내보내기는 로드맵에만 있고 아직은 지원 예정인 항목입니다.

### 모델 병합 및 배포 절차 (MXFP4 → vLLM)
1. **LoRA 어댑터 → MXFP4 병합**
   ```python
   from unsloth import FastLanguageModel

   model.save_pretrained_merged(
       output_dir="./artifacts/gpt_oss_mxfp4",
       tokenizer=tokenizer,
       save_method="mxfp4",   # GPT-OSS 전용 4bit 형식
   )
   ```
2. **vLLM Hot-swap 준비**
   - `workspace/model/gpt_oss/` 내부에 다음 파일이 존재해야 합니다.
     - `config.json`, `tokenizer.json`, `tokenizer.model`, `tokenizer_config.json`
     - `model.safetensors` (MXFP4 기반)
   - LoRA를 추가로 쓰려면 `adapter_config.json`, `adapter_model.safetensors`를 LoRA 폴더(`/workspace/model/gpt_oss/lora/`)에 둡니다.

3. **vLLM 서버 실행 (LoRA 핫스왑)**
   ```bash
   python -m vllm.entrypoints.openai.api_server \
       --model ./workspace/model/gpt_oss \
       --port 8000 \
       --enable-lora \
       --lora-modules ./workspace/model/lora/intent/ \
       --max-loras 8 \
       --max-lora-rank 64 \
       --gpu-memory-utilization 0.9
   ```

In [ ]:
model.save_pretrained("finetuned_model")
# model.push_to_hub("hf_username/finetuned_model", token = "hf_...") # Save to HF

### float16 또는 MXFP4 형식으로 저장하기

`float16` 또는 `MXFP4` 형식으로 직접 저장할 수도 있습니다. float16을 원하면 `merged_16bit`를 선택하세요.  
`push_to_hub_merged`를 사용하여 Hugging Face 계정에 업로드하세요!  
개인 토큰은 https://huggingface.co/settings/tokens 에서 확인할 수 있습니다.

In [ ]:
# mxfp4 4bit 형식으로 병합하고 허브에 푸시
if False:
    model.save_pretrained_merged("finetuned_model", tokenizer, save_method="mxfp4")
if False:
    model.push_to_hub_merged("repo_id/repo_name", tokenizer, token="hf...", save_method="mxfp4")

# 16bit 형식으로 병합하고 허브에 푸시
if False:
    model.save_pretrained_merged("finetuned_model", tokenizer, save_method="merged_16bit")
if False:  # HF Hub에 푸시
    model.push_to_hub_merged("hf/gpt-oss-finetune", tokenizer, save_method="merged_16bit", token="")

---

## 강화학습(Reinforcement Learning) 개요

Supervised Fine-Tuning(SFT) 이후 모델 성능을 한 단계 끌어올리는 강화학습 기법들을 소개합니다.

### RL 기법 비교표

| 기법 | 용도 | 데이터 형식 | 난이도 | 추천 상황 |
|------|------|------------|--------|----------|
| **GRPO** | 추론 능력 향상 | Question + Answer | 높음 | 수학, 코딩, 논리 문제 |
| **GSPO** | GRPO 변형 | Question + Answer | 높음 | GRPO 대안 |
| **DPO** | 선호도 학습 | Chosen + Rejected | 중간 | 응답 품질 개선 |
| **ORPO** | DPO 단순화 | Chosen + Rejected | 중간 | DPO보다 빠름 |
| **KTO** | 이진 피드백 | Good/Bad 라벨 | 낮음 | 간단한 평가 |

---

### 언제 어떤 기법을 사용할까?

```
SFT (Supervised Fine-Tuning)
    ↓
    기본 태스크 학습 완료
    ↓
┌───────────────────────────────────┐
│     목표가 무엇인가?              │
└───────────────────────────────────┘
    │
    ├─→ [추론 능력 향상] → GRPO/GSPO
    │   예: 수학 문제, 코딩, 논리 퍼즐
    │
    ├─→ [응답 품질 개선] → DPO/ORPO
    │   예: 더 자연스러운 대화, 안전성
    │
    └─→ [간단한 피드백] → KTO
        예: 좋음/나쁨 이진 평가
```

---

### GRPO (Group Relative Policy Optimization)

#### 핵심 아이디어
모델이 **추론 과정(Chain of Thought)**을 생성하도록 학습시켜, 복잡한 문제 해결 능력을 향상시킵니다.

#### 작동 원리
1. **여러 답변 생성**: 같은 질문에 대해 N개의 다른 답변 생성 (예: N=8)
2. **보상 함수 평가**: 각 답변의 품질을 점수화
3. **그룹 내 비교**: 같은 질문 내에서 좋은 답변은 강화, 나쁜 답변은 억제
4. **정책 업데이트**: 보상이 높은 답변 방향으로 모델 학습

#### 데이터셋 형식
```json
{
  "prompt": [
    {"role": "system", "content": "다음 형식으로 답하세요:\n<reasoning>...</reasoning>\n<answer>...</answer>"},
    {"role": "user", "content": "James는 사탕 20개를 가지고 있습니다. 친구 3명에게 각각 4개씩 주었다면, 몇 개가 남았나요?"}
  ],
  "answer": "8"  # 추론 과정 없이 정답만
}
```

#### 보상 함수 예시
```python
def reward_function(question, generated_response, ground_truth):
    score = 0
    
    # 1. 정답 여부 (+10점)
    if extract_answer(generated_response) == ground_truth:
        score += 10
    
    # 2. 추론 과정 존재 (+2점)
    if "<reasoning>" in generated_response:
        score += 2
    
    # 3. 단계별 설명 (+1점)
    reasoning_steps = count_steps(generated_response)
    score += min(reasoning_steps, 5)  # 최대 5단계
    
    # 4. 너무 긴 답변 (-1점)
    if len(generated_response) > 500:
        score -= 1
    
    return score
```

#### 학습 코드 예시
```python
from unsloth import FastLanguageModel
from trl import GRPOConfig, GRPOTrainer

# 모델 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.3-70B-Instruct",
    load_in_4bit=True,
)

# LoRA 적용
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)

# GRPO 설정
grpo_config = GRPOConfig(
    learning_rate=5e-6,            # RL은 낮은 학습률
    num_generations=8,             # 질문당 8개 답변 생성
    max_steps=1000,                # 학습 스텝
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    use_vllm=True,                 # vLLM 가속 사용
)

# Trainer 초기화
trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    train_dataset=dataset,
    reward_function=reward_function,  # 위에서 정의한 함수
    tokenizer=tokenizer,
)

# 학습 시작
trainer.train()
```

---

### GSPO (Group Sequence Policy Optimization)

GRPO의 변형으로, **sequence-level importance sampling**을 사용합니다.

#### GRPO vs GSPO
| 특성 | GRPO | GSPO |
|------|------|------|
| **샘플링 레벨** | Token-level | Sequence-level |
| **안정성** | 좋음 | 더 좋음 |
| **수렴 속도** | 보통 | 빠름 |
| **메모리 사용** | 같음 | 같음 |

#### 사용법
```python
grpo_config = GRPOConfig(
    importance_sampling_level="sequence",  # GSPO 활성화
    # 나머지는 GRPO와 동일
)
```

---

### DPO (Direct Preference Optimization)

#### 핵심 아이디어
사람의 선호도를 직접 학습: "이 답변이 저것보다 낫다"

#### 데이터셋 형식
```json
{
  "prompt": "파스타 만드는 법 알려줘",
  "chosen": "1. 물을 끓입니다\n2. 소금을 넣고 파스타를 삶습니다\n3. 8-10분 후 체에 받칩니다...",
  "rejected": "파스타 삶으면 돼요"
}
```

#### 학습 코드
```python
from unsloth import PatchDPOTrainer
from trl import DPOTrainer, DPOConfig

PatchDPOTrainer()

# 모델 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.1-8B-Instruct",
    load_in_4bit=True,
)

# LoRA 적용
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)

# DPO Trainer
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # Unsloth는 자동 처리
    args=DPOConfig(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        learning_rate=5e-6,
        num_train_epochs=3,
        logging_steps=1,
        optim="adamw_8bit",
        output_dir="outputs",
    ),
    beta=0.1,  # KL divergence 가중치
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    max_length=1024,
    max_prompt_length=512,
)

dpo_trainer.train()
```

---

### ORPO (Odds Ratio Preference Optimization)

DPO의 단순화 버전으로, **reference model 없이** 학습 가능.

#### 장점
- 메모리 효율 (Reference model 불필요)
- 학습 속도 빠름
- DPO와 유사한 성능

#### 사용법
```python
from trl import ORPOTrainer, ORPOConfig

orpo_trainer = ORPOTrainer(
    model=model,
    args=ORPOConfig(
        per_device_train_batch_size=4,
        learning_rate=8e-6,
        num_train_epochs=3,
        logging_steps=1,
        optim="adamw_8bit",
        output_dir="outputs",
    ),
    train_dataset=dpo_dataset,  # DPO와 같은 형식
    tokenizer=tokenizer,
)

orpo_trainer.train()
```

---

### KTO (Kahneman-Tversky Optimization)

가장 간단한 RL 기법: **이진 피드백**만 필요 (좋음/나쁨)

#### 데이터셋 형식
```json
{
  "prompt": "오늘 날씨 어때?",
  "completion": "오늘은 맑고 화창합니다!",
  "label": true  # true = 좋음, false = 나쁨
}
```

#### 사용법
```python
from trl import KTOTrainer, KTOConfig

kto_trainer = KTOTrainer(
    model=model,
    args=KTOConfig(
        per_device_train_batch_size=4,
        learning_rate=5e-6,
        num_train_epochs=1,
        logging_steps=1,
        optim="adamw_8bit",
        output_dir="outputs",
    ),
    train_dataset=kto_dataset,
    tokenizer=tokenizer,
)

kto_trainer.train()
```

---

### RL 학습 파이프라인 권장 순서

```
1단계: SFT (Supervised Fine-Tuning)
   ↓
   기본 태스크 능력 확보
   │
   ├─→ 2a단계: GRPO (추론 능력)
   │      ↓
   │   수학/코딩 문제 해결력 향상
   │
   └─→ 2b단계: DPO/ORPO (품질 개선)
          ↓
       응답 스타일, 안전성 조정
          ↓
       3단계: KTO (미세 조정)
          ↓
       사용자 피드백 반영
```

---

### RL 학습 팁

#### 1. Learning Rate는 낮게
```python
# 비권장: SFT 학습률 사용
learning_rate = 2e-4  # 너무 높음!

# 권장: RL 전용 학습률
learning_rate = 5e-6  # GRPO/DPO
learning_rate = 8e-6  # ORPO
```

#### 2. Beta 값 조정 (DPO/ORPO)
```python
# beta = KL divergence 가중치
beta = 0.1   # 표준 (원래 모델과 유사하게)
beta = 0.01  # 더 많이 변화 허용
beta = 0.5   # 보수적 (원래 모델 유지)
```

#### 3. Reward Hacking 방지
보상 함수를 악용하는 현상:
```python
# 나쁜 보상 함수
def bad_reward(response):
    return len(response)  # 길이만 보상 → 무한 반복 생성

# 좋은 보상 함수
def good_reward(response, answer):
    score = 0
    if extract_answer(response) == answer:
        score += 10  # 정확도 최우선
    if 50 < len(response) < 200:
        score += 2   # 적절한 길이
    return score
```

#### 4. 메모리 효율 최적화
```python
# Unsloth Memory-Efficient RL
grpo_config = GRPOConfig(
    use_vllm=True,  # vLLM 가속
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_completion_length=512,  # 너무 길지 않게
)
```

---

### 참고 자료
- **GRPO Tutorial**: [Train your own Reasoning model](https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/tutorial-train-your-own-reasoning-model-with-grpo)
- **Advanced RL Docs**: [Advanced Documentation](https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/advanced-rl-documentation)
- **DPO Paper**: [Direct Preference Optimization](https://arxiv.org/abs/2305.18290)
- **ORPO Paper**: [ORPO: Monolithic Preference Optimization](https://arxiv.org/abs/2403.07691)
- **Reward Hacking Guide**: [RL Reward Hacking](https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/rl-reward-hacking)

---

## 실습: GRPO로 추론 모델 만들기

이 섹션은 일반 모델을 GRPO로 추론 능력을 갖춘 모델로 변환하는 실습입니다.

### GRPO 데이터셋 준비

```python
from datasets import load_dataset, Dataset
import re

# 1. 시스템 프롬프트 정의
SYSTEM_PROMPT = """
다음 형식으로 답변하세요:

<reasoning>
단계별 사고 과정을 여기에 작성
</reasoning>

<answer>
최종 답변
</answer>
"""

# 2. 답변 추출 헬퍼 함수
def extract_xml_answer(text: str) -> str:
    """XML 형식에서 답변 추출"""
    if "<answer>" not in text:
        return text.strip()
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str:
    """#### 형식에서 답변 추출 (GSM8K용)"""
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# 3. GSM8K 데이터셋 준비
def prepare_gsm8k_dataset(split="train"):
    """수학 문제 데이터셋 준비"""
    data = load_dataset("openai/gsm8k", "main")[split]
    
    # 첫 1000개만 사용 (빠른 실습)
    data = data.select(range(min(1000, len(data))))
    
    def format_example(example):
        return {
            "prompt": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": example["question"]},
            ],
            "answer": extract_hash_answer(example["answer"]),
        }
    
    return data.map(format_example)

# 데이터셋 로드
dataset = prepare_gsm8k_dataset("train")
print(f"데이터셋 크기: {len(dataset)}")
print(f"첫 번째 예시:\n질문: {dataset[0]['prompt'][1]['content']}")
print(f"정답: {dataset[0]['answer']}")
```

### GRPO 보상 함수 정의

```python
import re

def grpo_reward_function(
    question: str,
    generated_response: str, 
    ground_truth_answer: str
) -> float:
    """
    GRPO 학습을 위한 보상 함수
    
    Returns:
        float: 보상 점수 (-10 ~ +20)
    """
    score = 0.0
    
    # 1. 정답 여부 확인 (가장 중요)
    try:
        generated_answer = extract_xml_answer(generated_response)
        
        # 숫자 비교 (수학 문제)
        gen_num = float(re.sub(r'[^\d.-]', '', generated_answer))
        truth_num = float(re.sub(r'[^\d.-]', '', ground_truth_answer))
        
        if abs(gen_num - truth_num) < 0.01:
            score += 10.0  # 정답 보상
        else:
            score -= 5.0   # 오답 페널티
    except:
        score -= 5.0
    
    # 2. 추론 과정 존재 여부
    if "<reasoning>" in generated_response and "</reasoning>" in generated_response:
        score += 3.0
    else:
        score -= 2.0  # 추론 없으면 페널티
    
    # 3. 추론 단계 수 (3~7단계가 이상적)
    reasoning_lines = generated_response.count("\n")
    if 3 <= reasoning_lines <= 10:
        score += 2.0
    elif reasoning_lines > 15:
        score -= 1.0  # 너무 장황함
    
    # 4. 답변 형식 준수
    if "<answer>" in generated_response and "</answer>" in generated_response:
        score += 1.0
    
    # 5. 길이 제한 (너무 짧거나 길면 안좋음)
    response_length = len(generated_response)
    if 100 < response_length < 800:
        score += 1.0
    elif response_length > 1500:
        score -= 2.0  # 너무 긺
    
    return score

# 테스트
test_response = """
<reasoning>
1. James는 사탕 20개를 가지고 있습니다.
2. 친구 3명에게 각각 4개씩 줍니다.
3. 총 준 사탕: 3 × 4 = 12개
4. 남은 사탕: 20 - 12 = 8개
</reasoning>

<answer>
8
</answer>
"""

print(f"보상 점수: {grpo_reward_function('test', test_response, '8')}")
```

### GRPO 학습 실행

```python
from unsloth import FastLanguageModel
from trl import GRPOConfig, GRPOTrainer

# 1. 모델 로드 (예시: Llama 3.1 8B)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# 2. LoRA 어댑터 추가
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

# 3. GRPO 설정
grpo_config = GRPOConfig(
    # 학습 파라미터
    learning_rate=5e-6,                 # RL용 낮은 학습률
    num_train_epochs=1,
    max_steps=100,                      # 실습용 짧게
    
    # 생성 파라미터
    num_generations=4,                  # 질문당 4개 답변 생성
    max_new_tokens=512,                 # 최대 생성 길이
    temperature=0.7,
    
    # 배치 파라미터
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    
    # RL 파라미터
    beta=0.0,                           # KL divergence (0=ref model 미사용)
    num_iterations=1,                   # PPO epochs
    
    # 최적화
    use_vllm=True,                      # vLLM 가속 사용
    optim="adamw_8bit",
    logging_steps=1,
    output_dir="./grpo_outputs",
    
    # Wandb (선택)
    report_to="none",  # "wandb"로 변경 시 추적
)

# 4. Trainer 초기화
trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    train_dataset=dataset,
    reward_function=grpo_reward_function,
    tokenizer=tokenizer,
)

# 5. 학습 시작
print("GRPO 학습 시작...")
trainer.train()
```

### 학습 후 테스트

```python
# 추론 모드로 전환
FastLanguageModel.for_inference(model)

# 테스트 질문
test_messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Emma는 연필 25자루를 가지고 있습니다. 친구들에게 7자루씩 3명에게 나눠줬다면, 몇 자루가 남았나요?"},
]

# 생성
inputs = tokenizer.apply_chat_template(
    test_messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True)

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=512,
    temperature=0.3,
    streamer=streamer,
)
```

### GRPO 하이퍼파라미터 튜닝 가이드

| 파라미터 | 기본값 | 조정 방향 |
|---------|--------|----------|
| **learning_rate** | `5e-6` | 발산하면 `1e-6`으로 낮추기 |
| **num_generations** | `4-8` | 메모리 충분하면 8로 증가 |
| **beta** | `0.0` | 과적합 시 `0.01~0.1` 시도 |
| **temperature** | `0.7` | 다양성 필요하면 `0.8~1.0` |
| **max_new_tokens** | `512` | 긴 추론 필요하면 `1024` |

### 보상 함수 개선 팁

```python
# 1. 도메인별 맞춤 보상
def math_reward(response, answer):
    # 수학: 정답 여부 + 풀이 과정
    pass

def code_reward(response, answer):
    # 코딩: 실행 가능 여부 + 테스트 통과
    pass

def writing_reward(response, answer):
    # 글쓰기: 문체, 길이, 구조
    pass

# 2. 외부 LLM 평가자 (GPT-4 등)
def llm_judge_reward(response, question):
    # ChatGPT API로 품질 평가
    pass

# 3. 복합 보상
def combined_reward(response, answer):
    score = 0
    score += correctness_reward(response, answer) * 0.6  # 정확도 60%
    score += reasoning_quality_reward(response) * 0.3     # 추론 30%
    score += format_reward(response) * 0.1                # 형식 10%
    return score
```

---

## 참고 자료

### Unsloth 공식 문서

#### 파인튜닝 기초
- **LoRA 하이퍼파라미터**: https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/lora-hyperparameters-guide
- **데이터셋 가이드**: https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/datasets-guide
- **지원 모델 목록**: https://docs.unsloth.ai/get-started/all-our-models
- **설치 가이드**: https://docs.unsloth.ai/get-started/installation

#### 강화학습 (RL)
- **GRPO 튜토리얼**: https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/tutorial-train-your-own-reasoning-model-with-grpo
- **Advanced RL 문서**: https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/advanced-rl-documentation
- **Reward Hacking 방지**: https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/rl-reward-hacking
- **Memory-Efficient RL**: https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/memory-efficient-rl
- **DPO/ORPO/KTO**: https://docs.unsloth.ai/get-started/reinforcement-learning-rl-guide/reinforcement-learning-dpo-orpo-and-kto

#### Colab 노트북(Unsloth)
- **GPT-OSS GRPO**: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb
- **Multiple Datasets**: https://colab.research.google.com/drive/1njCCbE1YVal9xC83hjdo2hiGItpY_D6t

### 트러블슈팅

#### Out of Memory (OOM)
```python
# 해결책 1: 배치 사이즈 줄이기
per_device_train_batch_size = 1
gradient_accumulation_steps = 8  # 실질 배치=8 유지

# 해결책 2: Gradient Checkpointing
use_gradient_checkpointing = "unsloth"

# 해결책 3: 더 작은 모델
model_name = "unsloth/Llama-3.2-3B-Instruct"  # 8B → 3B

# 해결책 4: 8bit 양자화
load_in_8bit = True  # 4bit 대신
```

#### 학습이 수렴하지 않음
```python
# 해결책 1: Learning Rate 조정
learning_rate = 1e-4  # 2e-4 → 1e-4

# 해결책 2: Warmup 추가
warmup_steps = 10

# 해결책 3: Scheduler 변경
lr_scheduler_type = "cosine"  # linear → cosine

# 해결책 4: Weight Decay 조정
weight_decay = 0.01
```

#### 과적합 (Overfitting)
```python
# 해결책 1: Epochs 줄이기
num_train_epochs = 1  # 3 → 1

# 해결책 2: Dropout 추가
lora_dropout = 0.05

# 해결책 3: 데이터 증강
# 합성 데이터 추가

# 해결책 4: 검증 세트 분리
dataset = dataset.train_test_split(test_size=0.1)
```

#### GRPO Reward Hacking
```python
# 해결책 1: 보상 함수 수정
# 길이 기반 보상 제거
# 정확도 중심으로

# 해결책 2: Beta 증가
beta = 0.1  # 0.0 → 0.1 (ref model 제약)

# 해결책 3: Temperature 조정
temperature = 0.5  # 0.7 → 0.5 (더 보수적)

# 해결책 4: 보상 정규화
# scale_rewards = "batch"
```

---

### Tips

#### 1. 학습 모니터링
```python
# Weights & Biases 사용
import wandb
wandb.init(project="my-finetuning")

# SFTConfig에 추가
report_to = "wandb"
logging_steps = 1
```

#### 2. 멀티 GPU 활용
```python
# 여러 GPU 사용
# accelerate 사용
accelerate launch --num_processes=4 train.py
```

---

### 학습 파이프라인 요약

```
┌─────────────────────────────────────┐
│   1. 태스크 정의                    │
│   (챗봇? 코딩? 추론? 번역?)         │
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│   2. 모델 선택                      │
│   (GPU 메모리, 성능 요구사항)       │
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│   3. 데이터 준비                    │
│   (최소 100개, 권장 1000+)          │
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│   4. SFT (Supervised Fine-Tuning)   │
│   (기본 능력 학습)                  │
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│   5. 평가 및 반복                   │
│   (성능 측정, 하이퍼파라미터 조정)  │
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│   6. RL (선택사항)                  │
│   (GRPO/DPO로 품질 향상)            │
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│   7. 배포                           │
│   (vLLM)                            │
└─────────────────────────────────────┘
```

---

### 추가 리소스

- **Unsloth GitHub**: https://github.com/unslothai/unsloth
- **Hugging Face / Unsloth 페이지**: https://huggingface.co/unsloth